## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
attrition_df.shape

(1470, 27)

In [3]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [5]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    "Education", 
    "Age", 
    "DistanceFromHome", 
    "JobSatisfaction", 
    "OverTime", 
    "StockOptionLevel", 
    "WorkLifeBalance", 
    "YearsAtCompany", 
    "YearsSinceLastPromotion", 
    "NumCompaniesWorked"
]


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=1)

In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
X_train["OverTime"].value_counts()

OverTime
No     798
Yes    304
Name: count, dtype: int64

In [8]:
X_train_encoded = pd.get_dummies(X_train, columns=['OverTime'], drop_first=True, dtype=int)
X_train_encoded = X_train_encoded.rename(columns={"OverTime_Yes" : "OverTime"})

X_test_encoded = pd.get_dummies(X_test, columns=['OverTime'], drop_first=True, dtype=int)
X_test_encoded = X_test_encoded.rename(columns={"OverTime_Yes" : "OverTime"})

In [9]:
X_train_encoded["OverTime"].value_counts()

OverTime
0    798
1    304
Name: count, dtype: int64

In [10]:
X_test_encoded["OverTime"].value_counts()

OverTime
0    256
1    112
Name: count, dtype: int64

In [11]:
X_train_encoded.head()

,Education,Age,DistanceFromHome,JobSatisfaction,StockOptionLevel,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,NumCompaniesWorked,OverTime
464,3,37,1,4,0,4,6,1,4,0
512,4,27,3,4,0,3,4,1,0,0
480,4,30,12,1,1,4,1,0,1,0
680,4,36,7,4,0,3,6,0,2,0
1023,2,56,1,1,1,4,3,1,2,0


In [12]:
# Create a StandardScaler
scaler = StandardScaler().fit(X_train_encoded)

# Fit the StandardScaler to the training data


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


In [13]:
y_df["Attrition"].value_counts()

Attrition
No     1233
Yes     237
Name: count, dtype: int64

In [14]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoded = department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])

department_train_encoded



array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [15]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoded = attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

attrition_train_encoded

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [16]:
# Find the number of columns in the X training data
X_train.shape

(1102, 10)

In [17]:
# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name='input')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='sigmoid', name='department_output')(department_hidden)


In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [20]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'binary_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 10)]                 0         []                            
                                                                                                  
 shared1 (Dense)             (None, 64)                   704       ['input[0][0]']               
                                                                                                  
 shared2 (Dense)             (None, 128)                  8320      ['shared1[0][0]']             
                                                                                                  
 department_hidden (Dense)   (None, 32)                   4128      ['shared2[0][0]']             
                                                                                             

In [21]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': department_train_encoded, 'attrition_output': attrition_train_encoded},
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10


28/28 [==============================] - 2s 12ms/step - loss: 1.1015 - department_output_loss: 0.5816 - attrition_output_loss: 0.5199 - department_output_accuracy: 0.6243 - attrition_output_accuracy: 0.8173 - val_loss: 1.0291 - val_department_output_loss: 0.5330 - val_attrition_output_loss: 0.4961 - val_department_output_accuracy: 0.6335 - val_attrition_output_accuracy: 0.8281
Epoch 2/10
28/28 [==============================] - 0s 3ms/step - loss: 0.8992 - department_output_loss: 0.4881 - attrition_output_loss: 0.4111 - department_output_accuracy: 0.6640 - attrition_output_accuracy: 0.8513 - val_loss: 0.9665 - val_department_output_loss: 0.5074 - val_attrition_output_loss: 0.4591 - val_department_output_accuracy: 0.6335 - val_attrition_output_accuracy: 0.8281
Epoch 3/10
28/28 [==============================] - 0s 3ms/step - loss: 0.8431 - department_output_loss: 0.4727 - attrition_output_loss: 0.3704 - department_output_accuracy: 0.6640 - attrition_output_accuracy: 0.8513 

In [22]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': department_test_encoded, 'attrition_output': attrition_test_encoded})
test_results

12/12 [==============================] - 0s 2ms/step - loss: 0.9236 - department_output_loss: 0.4927 - attrition_output_loss: 0.4309 - department_output_accuracy: 0.6413 - attrition_output_accuracy: 0.8342


[0.9236246943473816,
 0.4926884174346924,
 0.4309363067150116,
 0.6413043737411499,
 0.83423912525177]

In [23]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]}")
print(f"attrition Accuracy: {test_results[4]}")

Department Accuracy: 0.6413043737411499
attrition Accuracy: 0.83423912525177


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. I think accuracy is the best metric to use on attrition data because the results were either 1 or 0. Using accuracy for Department might not be the best to use because there is more that binary targets.
2. I only used sigmoid for the output layers because I could use either sigmoid or softmax for activation functions.
3. To improve the model I could use softmax on the department output layer since there are 3 target values. I could also change the accuracy metric to categorical accuracy since department is a multiclass target.